# Working with many files

In `1-singlefile.ipynb` we learned how to extract subsets and reproject a single image using a variety of tools (GDAL, rasterio, xarray, rioxarray, and holoviz). Often you want to work with a whole stack of imagery - for example let's see how to create a timeseries of backscatter over [Jakobshavn_Glacier](https://en.wikipedia.org/wiki/Jakobshavn_Glacier).

In [ ]:
# Bounding box of interest 
# draw on here: http://geojson.io/
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
gf = gpd.read_file('jakobshavn.geojson')
tiles = gv.tile_sources.EsriTerrain
bbox = gf.hvplot.polygons(alpha=0.2, geo=True)

lonmin, latmin, lonmax, latmax = gf.bounds.values[0]
print('bounding box=', lonmin, latmax, lonmax, latmin)

tiles * bbox

In [ ]:
# Use NASA CMR STAC endpoint to get files

# Can brows collecitons and catalogs by year
# https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS/collections/C1908075185-NSIDC_ECS/2015 

# Search returns a single-file STAC of all the items
#https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS/search?collections=C1908075185-NSIDC_ECS

import intake
from satsearch import Search

def get_STAC_items(url, collection, dates, bbox):
    results = Search.search(url=url,
                        collections=[collection], 
                        datetime=dates,
                        bbox=bbox,    
                        sortby=['+properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} Items')
    
    return intake.open_stac_item_collection(items)

In [ ]:
%%time
URL='https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS'
bbox = [lonmin, latmin, lonmax, latmax] 
collection = 'C1908075185-NSIDC_ECS' # MEaSUREs Greenland Image Mosaics from Sentinel-1A and -1B V003"
dates = '2014-01-01/2020-12-31' 
items = get_STAC_items(URL, collection, dates, bbox)

In [ ]:
# No great way to filter by asset type currently (gamma0, sigma0, thumbnail), so need a function
def filter_assets(catalog, key='1', pattern='gamma0'):
    
    all_assets = [item.assets.get(key) for item in catalog._stac_obj]
    all_hrefs = [asset.get('href') for asset in all_assets if asset] #ignore None values if asset key missing
    filtered = [i for i in all_hrefs if pattern in i]
    
    return filtered

pattern = 'gamma0'
asset_list = filter_assets(items, pattern='gamma0')
print(f'Found {len(asset_list)} urls matching pattern {pattern}')
asset_list[:3]

In [ ]:
# NOTE: reading from NSIDC SERVER REQUIRES you have a ~/.netrc file 
# behind the scenes we're using GDAL to make requests, and we set some Env vars for performance
#GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies

import os
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           GDAL_HTTP_COOKIEFILE='.urs_cookies',
           GDAL_HTTP_COOKIEJAR='.urs_cookies',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

In [ ]:
# Using COG overviews is a great way to get a quick low-resolution view of the data
import xarray as xr
import rioxarray
import hvplot.xarray
da = rioxarray.open_rasterio(asset_list[-1], overview_level=4, masked=True).squeeze('band') 
img = da.hvplot.image(cmap='gray', aspect='equal', frame_width=500)

# convert our bounding box to epsg:3413 (south polar sterographic)
gf3413 = gf.to_crs(3413)
aoi = gf3413.hvplot.polygons(alpha=0.2, color='red', aspect='equal', frame_width=500)

img *  aoi

In [ ]:
# In many cases we're only interested in a small subset of big data (like that bounding box above!)
#da.rio.clip(gf3413.geometry).hvplot.image(cmap='gray') #loads full raster
#da.rio.clip_box(**gf3413.bounds).hvplot.image(cmap='gray') # more memory-efficent

#new option in 0.2 release https://nbviewer.jupyter.org/github/corteva/rioxarray/blob/master/docs/examples/clip_geom.ipynb#Clipping-larger-rasters
#da = rioxarray.open(url, masked=True).rio.clip(geometries, from_disk=True)     

In [ ]:
%time

#open subset directly
da = rioxarray.open_rasterio(asset_list[0], masked=True).squeeze('band')
subset = da.rio.clip_box(**gf3413.bounds)

In [ ]:
%%time

# Will do this clipping taking advantage of multiple CPUs. In serial takes ~1.2 min
# NOTE: it should be same time to load full datasets or pre-clip with rio.clip (maybe not if using rio.clip from _disk)
# NOTE: using masked=True promotes float32 to float64 data


import dask
import pandas as pd

@dask.delayed
def lazy_open(href, masked=True):
    filename = href.split('/')[-1] 
    date = href.split('/')[-2] 
    da = rioxarray.open_rasterio(href, chunks=(1, "auto", -1), masked=masked).rename(band='time') 
    da['time'] = [pd.to_datetime(date)]
    da['filename'] = filename
    return da

# Seems single-machine scheduler uses threads by default (ThreadPool), you can use processes instead (ProcessPool)
with dask.config.set(scheduler='processes'): 
    dataArrays = dask.compute(*[lazy_open(href, masked=False) for href in asset_list])

In [ ]:
%%time

# NOTE: this is fast with dask arrays, can run out of memory with numpy arrays
DA = xr.concat(dataArrays, dim='time', join='override', combine_attrs='drop').rio.clip_box(**gf3413.bounds)
DA

In [ ]:
#%%time 

# Drop scenes that are all nans in the bbox

#test = DA.dropna('time', how='all') # da
#test

In [ ]:
#finally lets use a video scrubber widget
import panel as pn

# note: delay 1000ms between scenes
#player = pn.widgets.Player(name='time', loop_policy='once', interval=500)
#pn.Row(video, widgets={'time':player})

panel = DA.hvplot.image(x='x',y='y', 
                        rasterize=True,
                        cmap='gray', clim=(-25,5),
                        aspect='equal', frame_width=800,
                        widget_type='scrubber', widget_location='bottom') 

#widget = panel[1][1][0] 
#widget.interval = 500ms default
panel

In [ ]:
%%time 

subset = DA.to_dataset(name='gamma0').compute()
# save with compression
data_settings = {"zlib": True, "dtype":'float32', "complevel": 9}
encoding_dict = dict(gamma0=data_settings)
subset.to_netcdf('mysubset.nc', encoding=encoding_dict)